In [1]:
import re

import pandas as pd 
import numpy as np

import vectorisation as ex
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifierCV, RidgeClassifier
from sklearn.svm import SVC
from sklearn import metrics

In [2]:
Data = pd.read_csv('Shrec14/Data/Uli_data.csv')

In [3]:
Data[Data.freq==0]

,freq,trial,dim,birth,death
0,0,1,-2,63.3750,493.7600
1,0,1,-2,74.8260,493.7600
2,0,1,-1,47.0460,493.7600
3,0,1,0,47.0540,71.3880
4,0,1,0,48.0990,50.7160
...,...,...,...,...,...
871,0,9,1,3.7685,3.7704
872,0,9,1,3.7738,3.7794
873,0,9,1,3.7755,3.7858
874,0,9,1,3.7767,3.7822


In [4]:
Data.freq

0         0
1         0
2         0
3         0
4         0
         ..
210811    9
210812    9
210813    9
210814    9
210815    9
Name: freq, Length: 210816, dtype: int64

In [5]:
OneDgm = Data[Data.freq == 1]
OneDgm = OneDgm[OneDgm.trial == 5]
OneDgm.head()

,freq,trial,dim,birth,death
88669,1,5,-2,11.4570,36.0350
88670,1,5,-2,11.6030,36.0350
88671,1,5,-1,4.1397,36.0350
88672,1,5,0,4.1885,4.7006
88673,1,5,0,4.2057,4.7429


In [6]:
# Code to reshape the data in the groupby command below
def reshapeVec(g):
    A = np.array([g.dim,g.birth,g.death])
    A = A.T
    return A 

In [7]:
DgmsDF = Data.groupby(['freq', 'trial']).apply(reshapeVec)
DgmsDF = DgmsDF.reset_index()
DgmsDF = DgmsDF.rename(columns = {0:'CollectedDgm'})

In [8]:
DgmsDF.head()

,freq,trial,CollectedDgm
0,0,1,"[[-2.0, 63.375, 493.76], [-2.0, 74.826, 493.76..."
1,0,2,"[[-2.0, 42.301, 251.35], [-2.0, 44.127, 251.35..."
2,0,3,"[[-2.0, 24.084, 100.21], [-2.0, 24.433, 100.21..."
3,0,4,"[[-2.0, 16.221, 52.697], [-2.0, 16.226, 52.697..."
4,0,5,"[[-2.0, 11.783, 36.272], [-2.0, 11.93, 36.272]..."


In [9]:
def getDgm(A, dim = 0):
    if type(dim) != str:
        A = A[np.where(A[:,0] == dim)[0],1:]
    elif dim == 'essential':
        A = A[np.where(A[:,0] <0)[0],:]
    return(A)

In [10]:
DgmsDF['Dgm1'] = DgmsDF.CollectedDgm.apply(lambda x: getDgm(x, dim = 1))
DgmsDF['Dgm0'] = DgmsDF.CollectedDgm.apply(lambda x: getDgm(x, dim = 0))
DgmsDF['DgmInf'] = DgmsDF.CollectedDgm.apply(lambda x: getDgm(x, dim = 'essential'))

In [11]:
DgmsDF.head()

,freq,trial,CollectedDgm,Dgm1,Dgm0,DgmInf
0,0,1,"[[-2.0, 63.375, 493.76], [-2.0, 74.826, 493.76...","[[59.183, 244.09], [59.518, 244.29], [60.372, ...","[[47.054, 71.388], [48.099, 50.716], [48.465, ...","[[-2.0, 63.375, 493.76], [-2.0, 74.826, 493.76..."
1,0,2,"[[-2.0, 42.301, 251.35], [-2.0, 44.127, 251.35...","[[33.555, 251.35], [37.09, 37.137], [37.26, 37...","[[29.142, 38.969], [29.769, 30.07], [29.922, 3...","[[-2.0, 42.301, 251.35], [-2.0, 44.127, 251.35..."
2,0,3,"[[-2.0, 24.084, 100.21], [-2.0, 24.433, 100.21...","[[15.836, 100.21], [19.13, 99.232], [19.36, 71...","[[14.26, 19.491], [14.269, 14.397], [14.356, 1...","[[-2.0, 24.084, 100.21], [-2.0, 24.433, 100.21..."
3,0,4,"[[-2.0, 16.221, 52.697], [-2.0, 16.226, 52.697...","[[7.8871, 52.697], [9.4849, 44.929], [9.659, 4...","[[7.2928, 9.4729], [7.3087, 7.3396], [7.3128, ...","[[-2.0, 16.221, 52.697], [-2.0, 16.226, 52.697..."
4,0,5,"[[-2.0, 11.783, 36.272], [-2.0, 11.93, 36.272]...","[[4.7617, 36.272], [4.8185, 36.261], [4.8253, ...","[[4.3036, 4.3046], [4.3337, 4.7526], [4.431, 4...","[[-2.0, 11.783, 36.272], [-2.0, 11.93, 36.272]..."


In [12]:
def label(index):
    if 0 <= index <= 19:
        return 'male_neutral'
    elif 20<= index <=39:
        return 'male_bodybuilder'
    elif 40<= index <=59:
        return 'male_fat'
    elif 60<= index <=79:
        return 'male_thin'
    elif 80<= index <=99:
        return 'male_average'
    elif 100<= index <=119:
        return 'female_neutral'
    elif 120<= index <=139:
        return 'female_bodybuilder'
    elif 140<= index <=159:
        return 'female_fat'
    elif 160<= index <=179:
        return 'female_thin'
    elif 180<= index <=199:
        return 'female_average'
    elif 200<= index <=219:
        return 'child_neutral'
    elif 220<= index <=239:
        return 'child_bodybuilder'
    elif 240<= index <=259:
        return 'child_fat'
    elif 260<= index <=279:
        return 'child_thin'
    elif 280<= index <=299:
        return 'child_average'
    else:
        print('What are you giving me?')

In [13]:
DgmsDF['TrainingLabel'] = DgmsDF.freq.apply(label)
DgmsDF= DgmsDF.sample(frac=1)
DgmsDF.head()

,freq,trial,CollectedDgm,Dgm1,Dgm0,DgmInf,TrainingLabel
2378,237,9,"[[-2.0, 3.7595, 3.7905], [-2.0, 3.7602, 3.7905...","[[3.7417, 3.7905], [3.749, 3.7605], [3.7502, 3...","[[3.7416, 3.7417], [3.7416, 3.7416], [3.7416, ...","[[-2.0, 3.7595, 3.7905], [-2.0, 3.7602, 3.7905...",child_bodybuilder
880,88,1,"[[-2.0, 62.486, 488.5], [-2.0, 72.413, 488.5],...","[[53.977, 53.98], [54.261, 54.268], [54.306, 2...","[[41.828, 69.22], [45.729, 68.662], [46.953, 6...","[[-2.0, 62.486, 488.5], [-2.0, 72.413, 488.5],...",male_average
2955,295,6,"[[-2.0, 8.9505, 24.702], [-2.0, 9.3618, 24.702...","[[4.2419, 21.402], [5.1263, 24.139], [5.1733, ...","[[4.1727, 4.2323]]","[[-2.0, 8.9505, 24.702], [-2.0, 9.3618, 24.702...",child_average
864,86,5,"[[-2.0, 11.871, 35.335], [-2.0, 11.896, 35.335...","[[4.8306, 27.249], [4.9837, 35.221], [5.003, 3...","[[4.1402, 4.1469], [4.7265, 4.7645], [11.562, ...","[[-2.0, 11.871, 35.335], [-2.0, 11.896, 35.335...",male_average
1338,133,9,"[[-2.0, 1.6026, 3.0279], [-2.0, 1.6117, 3.0279...","[[1.2921, 3.0279], [1.3639, 2.9276], [1.5284, ...","[[1.2888, 1.2912], [1.2888, 1.2888], [1.2889, ...","[[-2.0, 1.6026, 3.0279], [-2.0, 1.6117, 3.0279...",female_bodybuilder


# Features & Classification

In [14]:
func_list = [ex.GetPersStats,
             ex.GetPersLifespanFeature,
             ex.GetPersImageFeature,
             ex.GetPersLandscapeFeature,
             ex.GetPersEntropyFeature,
             ex.GetBettiCurveFeature,
             ex.GetCarlssonCoordinatesFeature,
             ex.GetPersSilhouetteFeature,
             ex.GetTopologicalVectorFeature,
             # ex.GetAtolFeature,
             ex.GetComplexPolynomialFeature,
             ex.GetPersLifespanFeature,
             ex.GetPersTropicalCoordinatesFeature
            ]

In [15]:
def compute_feature(func,Xs):
    d = dict()
    ls = range(0,len(Xs))
    for i in ls:
        d[str(i)]=Xs[i]
    feature_Xs = dict()
    for i in ls:
        feature_Xs[str(i)]=func(d[str(i)]) 
    Xs=[feature_Xs[str(i)] for i in ls]
    return Xs

In [16]:
def rforest_1(trial,dim,func):
    dgms_data=list(DgmsDF["Dgm"+str(dim)][DgmsDF["trial"]==trial])
    dgms_labels=list(DgmsDF["TrainingLabel"][DgmsDF["trial"]==trial])
    le = preprocessing.LabelEncoder()
    le.fit(dgms_labels)
    dgms_labels_numeric=le.transform(dgms_labels)
    X_train, X_test, y_train, y_test = train_test_split(dgms_data, dgms_labels_numeric, test_size=0.30)
    X_train = compute_feature(func,X_train)
    X_test = compute_feature(func,X_test)
    clf = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
    clf.fit(X_train,y_train)
    score = clf.score(X_test , y_test)
    return score

In [17]:
scores = list()
for i in range(1,9):
    scores.append(rforest_1(i,0,ex.GetPersStats))
print(np.max(scores))

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:250: RuntimeWarning: invalid value encountered in true_divide
  p = p/np.sum(p)
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:250: RuntimeWarning: invalid value encountered in true_divide
  p = p/np.sum(p)
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenc

0.6888888888888889


/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

In [18]:
scores

[0.5333333333333333,
 0.45555555555555555,
 0.25555555555555554,
 0.37777777777777777,
 0.3111111111111111,
 0.3888888888888889,
 0.5444444444444444,
 0.6888888888888889]

In [19]:
scores = list()
for i in range(1,9):
    scores.append(rforest_1(i,1,ex.GetPersStats))
print(np.max(scores))

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

0.8888888888888888


/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))


In [20]:
for dim in ["Dgm0","Dgm1"]:
    results_cv_5 = dict()
    results_70_30_dgm1_rf = dict()
    scores = list()
    for n in range(1,9):
        dgms_data=list(DgmsDF["Dgm1"][DgmsDF["trial"]==n])
        dgms_labels=list(DgmsDF["TrainingLabel"][DgmsDF["trial"]==n])
        le = preprocessing.LabelEncoder()
        le.fit(dgms_labels)
        dgms_labels_numeric=le.transform(dgms_labels)
        X_train, X_test, y_train, y_test = train_test_split(dgms_data, dgms_labels_numeric, test_size=0.30)
        # train features
        d_train = dict()
        for i in range(0,len(X_train)):
            d_train[str(i)]=X_train[i]
        features_train = dict()
        for i in range(0,len(X_train)):
            features_train[str(i)]=func(d_train[str(i)]) 
        X_train=[features_train[str(i)] for i in range(0,len(X_train))]
        # test features
        d_test = dict()
        for i in range(0,len(X_test)):
            d_test[str(i)]=X_test[i]
        features_test = dict()
        for i in range(0,len(X_test)):
            features_test[str(i)]=func(d_test[str(i)]) 
        X_test=[features_test[str(i)] for i in range(0,len(X_test))]
        #
        # Random forest 70 trainin 30 test
        clf = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
        clf.fit(X_train,y_train)
        score = clf.score(X_test , y_test)
        scores.append(score)
    fs[func.__name__] = scores
    for func in func_list:
        results_70_30_dgm1_rf[dim+"_"+func.__name__]=np.max(fs[func.__name__])
        
        
        
        

NameError: name 'func' is not defined

In [ ]:
fs = dict()
results_cv_5 = dict()
for func in func_list:
    scores = list()
    for n in range(1,9):
        dgms_train=list(DgmsDF["Dgm1"][DgmsDF["trial"]==n])
        dgms_labels=list(DgmsDF["TrainingLabel"][DgmsDF["trial"]==n])
        le = preprocessing.LabelEncoder()
        le.fit(dgms_labels)
        dgms_labels_numeric=le.transform(dgms_labels)
        d = dict()
        for i in range(0,len(dgms_train)):
            d[str(i)]=dgms_train[i]
        features = dict()
        for i in range(0,len(dgms_train)):
            features[str(i)]=func(d[str(i)]) # [:-1] to remove infinity bar

        X=[features[str(i)] for i in range(300)]
        clf = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
        score = cross_val_score(clf,X , dgms_labels_numeric, cv=5)
        scores.append(score.mean())
        fs[func.__name__] = scores
for func in func_list:
    results_cv_5[func.__name__]=np.max(fs[func.__name__])

In [ ]:
results_cv_5

70% training, 30% test

In [ ]:
fs = dict()
results_70_30_dgm1_rf = dict()
for func in func_list:
    scores = list()
    for n in range(1,9):
        dgms_data=list(DgmsDF["Dgm1"][DgmsDF["trial"]==n])
        dgms_labels=list(DgmsDF["TrainingLabel"][DgmsDF["trial"]==n])
        le = preprocessing.LabelEncoder()
        le.fit(dgms_labels)
        dgms_labels_numeric=le.transform(dgms_labels)
        X_train, X_test, y_train, y_test = train_test_split(dgms_data, dgms_labels_numeric, test_size=0.30)
        # X train features
        d_train = dict()
        for i in range(0,len(X_train)):
            d_train[str(i)]=X_train[i]
        features_train = dict()
        for i in range(0,len(X_train)):
            features_train[str(i)]=func(d_train[str(i)]) 
        X=[features_train[str(i)] for i in range(0,len(X_train))]
        # test features
        d_test = dict()
        for i in range(0,len(X_test)):
            d_test[str(i)]=X_test[i]
        features_test = dict()
        for i in range(0,len(X_test)):
            features_test[str(i)]=func(d_test[str(i)]) 
        X_test=[features_test[str(i)] for i in range(0,len(X_test))]
        
        clf = RandomForestClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0)
        clf.fit(X,y_train)
        score = clf.score(X_test , y_test)
        scores.append(score)
        fs[func.__name__] = scores
for func in func_list:
    results_70_30_dgm1_rf[func.__name__]=np.max(fs[func.__name__])

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:252: RuntimeWarning: divide by zero encountered in log
  ent = -np.dot(p, np.log(p))
/home/cimagroup/vectorisation-maps/vectorisation/Get

...

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered

/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered in log
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: invalid value encountered in double_scalars
  ent[min_idx:max_idx]-=p[j]*np.log(p[j])
/home/cimagroup/vectorisation-maps/vectorisation/GetPersistenceCurves.py:261: RuntimeWarning: divide by zero encountered

In [ ]:
results_70_30_dgm1_rf